In [ ]:
from sympy import Matrix, Symbol, simplify, symbols


def build_functional_matrix(n, k) -> Matrix:
    f = Matrix(symbols(",".join(f"f{j}" for j in range(1, n + 1))))
    gammas = symbols(",".join(f"\gamma{i}" for i in range(1, k + 1)))
    lambdas = symbols(",".join(f"\lambda{i}" for i in range(1, k + 1)))
    # lambdas = [-float(i + 1.) * 0.1 for i in range(1, k + 1)]
    # print(lambdas)
    result = Matrix.zeros(rows=n, cols=1)
    if k == 1:
        gammas = [gammas]
        lambdas = [lambdas]
    for gamma, lambd in zip(gammas, lambdas):
        result += gamma * Matrix([lambd**j for j in range(n)])

    diff: Matrix = f - result
    return diff


def build_functional(n, k):
    diff = build_functional_matrix(n, k)
    return simplify(diff.norm() ** 2)


In [ ]:
build_functional(4, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import Math, display
from scipy.optimize import minimize
from sympy.printing import latex
from toolz import compose

mega_print = compose(display, Math) if False else print

n = 6

while True:
    f_vec = np.round(np.random.randint(-3, 3, n) + np.random.random(n), 2)
    # 6 2 10.226062424586425 gamma=[0.5823549  0.58430421] lambda=[-0.79111583 -1.17055549]
    # print(f_vec)
    mega_print(f"F={latex(Matrix(f_vec))}")
    index, function_result = [], []
    for k in range(1, n - 1):
        J = build_functional(n, k)
        # display(J)
        expr = J.subs({Symbol(f"f{i}"): f_vec[i - 1] for i in range(1, n + 1)})

        def func(x):
            return float(
                expr.subs(
                    {Symbol(f"\gamma{i + 1}"): x[i] for i in range(0, k)}
                    | {Symbol(f"\lambda{i + 1 - k}"): x[i] for i in range(k, 2 * k)}
                ).evalf(n=21)
            )

        b_gamma = [-1e6, 1e6]
        b_lambda = [-1e6, -1]
        bounds = [b_gamma] * k + [b_lambda] * k

        x0 = [-1] * k + [-it - 1 for it in range(k)]
        solution = minimize(
            func,
            x0,
            bounds=bounds,  # method="SLSQP"
        )  # method="L-BFGS-B", tol=1e-9)
        # print(lambdas)
        gammas, lambdas = solution.x[:k], solution.x[k:]
        mega_print(f"k={k}", end=" & ")
        mega_print(f"\min{{J_{k}^{n}}}={np.round(solution.fun, 2)}", end=" & ")
        for i, x in enumerate(solution.x[:k]):
            mega_print(f"\gamma_{i + 1}={np.round(x, 2)}", end=", ")
        print(" & ", end="")
        for i, x in enumerate(solution.x[k:]):
            mega_print(f"\lambda_{i + 1}={np.round(x, 2)}", end=", ")

        gramm_matrix = np.zeros((k, k))
        for i, l_i in enumerate(lambdas):
            li = np.array([l_i**k for k in range(n)])
            for j, l_j in enumerate(lambdas):
                lj = np.array([l_j**k for k in range(n)])
                gramm_matrix[i, j] = li @ lj
        print(" & ", end="")
        mega_print(
            f"eig(G_{k}^{n})={np.round(np.linalg.eigvals(gramm_matrix), 5)}",
            end="\\\\ \hline",
        )

        print()
        index.append(k)
        function_result.append(np.round(solution.fun, 2))

    fig = go.Figure(go.Scatter(x=index, y=function_result))
    fig.update_layout(
        xaxis_title="Значение 'k'",
        yaxis_title="min J",
        plot_bgcolor="white",
    )
    fig.update_xaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
    )
    fig.update_yaxes(
        mirror=True,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="lightgrey",
    )
    fig.show()


In [ ]:
import numpy as np


def generate_data(n, k_true, noise_level=0.1):
    """
    Генерирует данные согласно вашей модели с k_true компонентами
    """
    # Генерируем истинные параметры
    l_true = -np.random.exponential(1, k_true)  # отрицательные l_i
    g_coeffs = np.random.normal(0, 1, k_true)  # коэффициенты g_i

    # Создаем матрицу базисных функций
    t = np.arange(n)
    basis_matrix = np.array([l**t for l in l_true]).T

    # Генерируем целевой вектор f
    f_true = basis_matrix @ g_coeffs
    f_noisy = f_true + np.random.normal(0, noise_level, n)

    return f_noisy, l_true, g_coeffs


generate_data(6, 3)